In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('./'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

from keras.models import Model, load_model
from keras.layers import Input, Dropout, Dense, Embedding, SpatialDropout1D, concatenate, BatchNormalization, Flatten
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text, sequence
from keras.callbacks import Callback
from keras import backend as K
from keras.models import Model
from keras.losses import mean_squared_error as mse_loss

from keras import optimizers
from keras.optimizers import RMSprop, Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

./sample_submission.csv
./submission_withoutleak001.csv.gz
./submission_nn007lofo.csv.gz
./generate_datasets.ipynb
./rows_to_drop.csv
./building_all_meters.feather
./train_with_cnt_per_tmp_meter.feather
./test_simple_cleanup.feather
./site4.csv
./winning sub.ipynb
./level2--ensembling_model.ipynb
./untitled.txt
./submission_multimeter003.csv.gz
./tests_building.ipynb
./level1--submission_multimeter003.ipynb
./site1.pkl
./level1--submission_whatsyourcv3_0052_trncl.ipynb
./weather_train.csv
./generate_leak_data.ipynb
./leak012345_001.feather
./site15_leakage.csv
./train.csv
./submission_multimeter004_nobuild.csv.gz
./model_0.hdf5
./test.csv
./weather_test.csv
./submission_ucf_replaced.csv
./level1--submission_nn007lofo--CNN.ipynb
./level1--submission_nn001-DenseNN.ipynb
./submission_whatsyourcv3_0052_trncl.csv.gz
./building_metadata.csv
./asu_2016-2018.csv.zip
./train_cleanup_001.feather
./train_simple_cleanup.feather
./level1--submission_multimeter004_nobuild.ipynb
./level1--submission_

Using TensorFlow backend.
/home/ubuntu/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu/anaconda3/lib/python3.6/site-packages/tenso

In [2]:
train = pd.read_feather("./train_simple_cleanup.feather")
train.columns

Index(['building_id', 'meter', 'timestamp', 'site_id', 'primary_use',
       'square_feet', 'year_built', 'floor_count', 'na_year_built',
       'na_floor_count', 'build_na_total', 'meter_reading_0',
       'meter_reading_1', 'meter_reading_2', 'meter_reading_3',
       'cnt_building_per_site', 'cnt_building_per_site_prim',
       'sqr_mean_per_site', 'sqr_mean_per_prim_site', 'air_temperature',
       'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr',
       'sea_level_pressure', 'wind_direction', 'wind_speed', 'RH', 'heat',
       'windchill', 'feellike', 'air_temperature_mean_lag3',
       'dew_temperature_mean_lag3', 'heat_mean_lag3', 'windchill_mean_lag3',
       'feellike_mean_lag3', 'had_air_temperature', 'had_cloud_coverage',
       'had_dew_temperature', 'had_precip_depth_1_hr',
       'had_sea_level_pressure', 'had_wind_direction', 'had_wind_speed',
       'had_RH', 'had_heat', 'had_windchill', 'had_feellike',
       'had_air_temperature_mean_lag3', 'had_dew_temperatur

In [3]:
from sklearn.preprocessing import LabelEncoder

# le = LabelEncoder()
# train["primary_use"] = le.fit_transform(train["primary_use"])

categoricals = ["site_id", "building_id", "primary_use", "tm_hour_of_day", "tm_day_of_week",  "meter"]

drop_cols = ["sea_level_pressure", "wind_speed", "wind_direction"]


fonct_cols_list = ['meter_reading_0',
       'meter_reading_1', 'meter_reading_2', 'meter_reading_3',
       'cnt_building_per_site', 'cnt_building_per_site_prim',
       'sqr_mean_per_site', 'sqr_mean_per_prim_site',
    'RH', 'heat',
       'windchill', 'feellike', 'air_temperature_mean_lag3',
       'dew_temperature_mean_lag3', 'heat_mean_lag3', 'windchill_mean_lag3',]

numcols_3d = ["tm_hour_of_day", 'RH', 'heat', 'windchill', 'feellike',
             "air_temperature", "cloud_coverage","dew_temperature", "precip_depth_1_hr"]

had_cols_list = ['had_air_temperature', 'had_cloud_coverage',
       'had_dew_temperature', 'had_precip_depth_1_hr',
       'had_sea_level_pressure', 'had_wind_direction', 'had_wind_speed',
       'had_RH', 'had_heat', 'had_windchill', 'had_feellike',
       'had_air_temperature_mean_lag3', 'had_dew_temperature_mean_lag3',
       'had_heat_mean_lag3', 'had_windchill_mean_lag3',
       'had_feellike_mean_lag3']

numericals = ["square_feet", "year_built", "air_temperature", "cloud_coverage",
              "dew_temperature", "precip_depth_1_hr", "floor_count", ]+drop_cols

feat_cols = categoricals + numericals + fonct_cols_list + had_cols_list

In [4]:
len(numcols_3d)

9

In [5]:
train['square_feet'] = train['square_feet'].apply(lambda x: int(x/1000))

In [6]:
target = train["meter_reading"]#np.log1p(train["meter_reading"])

del train["meter_reading"] 

# train = train.drop(drop_cols, axis = 1)

In [7]:
max_col = {}
for col in numericals+ fonct_cols_list:
    if col != 'meter_reading':
        max_ = max(-train[col].min(), train[col].max())
        train[col] = train[col] / max_
        max_col[col]=max_
max_col

{'square_feet': 875,
 'year_built': 2017,
 'air_temperature': 47.20000076293945,
 'cloud_coverage': 9.0,
 'dew_temperature': 35.0,
 'precip_depth_1_hr': 343.0,
 'floor_count': 26,
 'sea_level_pressure': 1045.5,
 'wind_speed': 19.0,
 'wind_direction': 360.0,
 'meter_reading_0': 1,
 'meter_reading_1': 1,
 'meter_reading_2': 1,
 'meter_reading_3': 1,
 'cnt_building_per_site': 274,
 'cnt_building_per_site_prim': 92,
 'sqr_mean_per_site': 290625.0,
 'sqr_mean_per_prim_site': 405083.0,
 'RH': 158.5,
 'heat': 225.71316528320312,
 'windchill': 1844.1282958984375,
 'feellike': 225.71316528320312,
 'air_temperature_mean_lag3': 47.20000076293945,
 'dew_temperature_mean_lag3': 34.766666412353516,
 'heat_mean_lag3': 217.25637817382812,
 'windchill_mean_lag3': 1837.2135009765625}

In [8]:
train.columns

Index(['building_id', 'meter', 'timestamp', 'site_id', 'primary_use',
       'square_feet', 'year_built', 'floor_count', 'na_year_built',
       'na_floor_count', 'build_na_total', 'meter_reading_0',
       'meter_reading_1', 'meter_reading_2', 'meter_reading_3',
       'cnt_building_per_site', 'cnt_building_per_site_prim',
       'sqr_mean_per_site', 'sqr_mean_per_prim_site', 'air_temperature',
       'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr',
       'sea_level_pressure', 'wind_direction', 'wind_speed', 'RH', 'heat',
       'windchill', 'feellike', 'air_temperature_mean_lag3',
       'dew_temperature_mean_lag3', 'heat_mean_lag3', 'windchill_mean_lag3',
       'feellike_mean_lag3', 'had_air_temperature', 'had_cloud_coverage',
       'had_dew_temperature', 'had_precip_depth_1_hr',
       'had_sea_level_pressure', 'had_wind_direction', 'had_wind_speed',
       'had_RH', 'had_heat', 'had_windchill', 'had_feellike',
       'had_air_temperature_mean_lag3', 'had_dew_temperatur

In [10]:


# https://www.kaggle.com/divrikwicky/lightweight-version-of-2-65-custom-nn

def nn_block(input_layer, size, dropout_rate, activation):
    out_layer = Dense(size, activation=None)(input_layer)
    out_layer = BatchNormalization()(out_layer)
    out_layer = Activation(activation)(out_layer)
    out_layer = Dropout(dropout_rate)(out_layer)
    return out_layer

def cnn_block(input_layer, size, dropout_rate, activation):
    out_layer = Conv1D(size, 1, activation=None)(input_layer)
    out_layer = BatchNormalization()(out_layer)
    out_layer = Activation(activation)(out_layer)
    out_layer = Dropout(dropout_rate)(out_layer)
    return out_layer

def model_lofo(dense_dim_1=64, dense_dim_2=32, dense_dim_3=32, dense_dim_4=16, 
            dropout1=0.2, dropout2=0.1, dropout3=0.1, dropout4=0.1, lr=0.001):

    #Inputs
    site_id = Input(shape=[1], name="site_id")
    building_id = Input(shape=[1], name="building_id")
    meter = Input(shape=[1], name="meter")
    primary_use = Input(shape=[1], name="primary_use")
    square_feet = Input(shape=[1], name="square_feet")
    year_built = Input(shape=[1], name="year_built")
    air_temperature = Input(shape=[1], name="air_temperature")
    cloud_coverage = Input(shape=[1], name="cloud_coverage")
    dew_temperature = Input(shape=[1], name="dew_temperature")
    tm_hour_of_day = Input(shape=[1], name="tm_hour_of_day")
    precip = Input(shape=[1], name="precip_depth_1_hr")
    tm_day_of_week = Input(shape=[1], name="tm_day_of_week")
#     beaufort_scale = Input(shape=[1], name="beaufort_scale")

    orig_feature = Input(shape=[len(numericals)], name="orig_feature")
    had_cols = Input(shape=[len(had_cols_list)], name="had_cols")
    fonct_cols = Input(shape=[len(fonct_cols_list)], name="fonct_cols")
    
    all_num3D = Input(shape=(train[numcols_3d].shape[1], 3), name="all_num3D")
   
    #Embeddings layers
    emb_site_id = Embedding(16, 2)(site_id)
    emb_building_id = Embedding(1449, 3)(building_id)
    emb_meter = Embedding(4, 2)(meter)
    emb_primary_use = Embedding(16, 2)(primary_use)
#     emb_hour = Embedding(24, 2)(tm_hour_of_day)
#     emb_weekday = Embedding(7, 2)(tm_day_of_week)

    concat_emb = concatenate([
           Flatten() (emb_site_id)
         , Flatten() (emb_building_id)
         , Flatten() (emb_primary_use)
#          , Flatten() (emb_hour)
#          , Flatten() (emb_weekday)
    ])
    
#     categ = Dropout(dropout1)(Dense(dense_dim_1,activation='relu') (concat_emb))
#     categ = BatchNormalization()(categ)
#     categ = Dropout(dropout2)(Dense(dense_dim_2,activation='relu') (categ))
    
    #main layer
    main_l = concatenate([
#           categ
        square_feet
        , year_built
        , air_temperature
        , cloud_coverage
        , dew_temperature
        , tm_hour_of_day
        , tm_day_of_week
        , precip
#         , beaufort_scale
    ])
    dense_input = concatenate([orig_feature, concat_emb, had_cols, fonct_cols])

#     type_input = Input(shape=(1,))
#     type_emb = Flatten()(Embedding(dev_df["le_type"].max() + 1, 5)(type_input))

    type_emb = Flatten()(emb_meter)
    
    mol_input = all_num3D #RepeatVector(3)(dense_input) #Input(shape=(M_train.shape[1], M_train.shape[2]))
    mol_layer = cnn_block(mol_input, 200, 0.05, "relu")
    mol_layer = cnn_block(mol_layer, 100, 0.05, "relu")
    
    merged_input = BatchNormalization()(concatenate([dense_input, type_emb, 
                                                     GlobalMaxPooling1D()(mol_layer), GlobalAvgPool1D()(mol_layer)]))
    
    x1 = Dense(200, activation="relu")(merged_input)
    x2 = nn_block(x1, 20, 0.01, "sigmoid")
    
    mol_layer = concatenate([RepeatVector(9)(x2), mol_layer])
    mol_layer = cnn_block(mol_layer, 200, 0.05, "relu")
    mol_layer = cnn_block(mol_layer, 100, 0.05, "relu")
    merged_input = BatchNormalization()(concatenate([dense_input, type_emb, 
                                                     GlobalMaxPooling1D()(mol_layer), GlobalAvgPool1D()(mol_layer)]))
    hidden_layer = concatenate([Dense(600, activation="relu")(merged_input), x1])
    
    hidden_layer = nn_block(hidden_layer, 400, 0.05, "relu")
    hidden_layer = nn_block(hidden_layer, 200, 0.05, "relu")
    hidden_layer = nn_block(hidden_layer, 100, 0.05, "relu")
    
    hidden_layer = concatenate([Flatten()(emb_meter), hidden_layer])
    
    output = Dense(1, activation="linear")(hidden_layer)


    model = Model([ site_id,
                    building_id, 
                    meter, 
                    primary_use, 
                    square_feet, 
                    year_built, 
                   
                    air_temperature,
                    cloud_coverage,
                    dew_temperature, 
                    tm_hour_of_day,
                    tm_day_of_week, 
                   precip,
                   
                   all_num3D,
                   orig_feature,
                   had_cols,
                   fonct_cols,
                    
                    ], output)

    model.compile(optimizer = Nadam(lr=lr),
                  loss= mse_loss,
                  metrics=[root_mean_squared_error])
    return model

In [11]:
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=0))

In [12]:
from pandas.api.types import is_datetime64_any_dtype as is_datetime
from pandas.api.types import is_categorical_dtype

def reduce_mem_usage(df, use_float16=False):
    """
    Iterate through all the columns of a dataframe and modify the data type to reduce memory usage.        
    """
    
    start_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage of dataframe is {:.2f} MB".format(start_mem))
    
    for col in df.columns:
        if is_datetime(df[col]) or is_categorical_dtype(df[col]):
            continue
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if use_float16 and c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype("category")

    end_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage after optimization is: {:.2f} MB".format(end_mem))
    print("Decreased by {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [18]:
# introduction of lagged features (-1 hours and -2 hours)

train = reduce_mem_usage(train)

train1 = train.groupby(['building_id', 'meter'])[numcols_3d].shift(1).fillna(0)
train1 = reduce_mem_usage(train1)
train2 = train.groupby(['building_id', 'meter'])[numcols_3d].shift(2).fillna(0)
train2 = reduce_mem_usage(train2)

Memory usage of dataframe is 2486.35 MB
Memory usage after optimization is: 2486.35 MB
Decreased by 0.0%
Memory usage of dataframe is 731.28 MB
Memory usage after optimization is: 658.15 MB
Decreased by 10.0%
Memory usage of dataframe is 731.28 MB
Memory usage after optimization is: 658.15 MB
Decreased by 10.0%


In [21]:
# fct to return the data as 3D table embeded into a dictionnary:
def get_keras_data(df, df1, df2, num_cols, cat_cols):
    cols = cat_cols + num_cols
    X = {col: np.array(df[col]) for col in cols}
    X['orig_feature'] = np.array(df[numericals])
    X['had_cols'] = np.array(df[had_cols_list])
    X['fonct_cols'] = np.array(df[fonct_cols_list])
    
    X['all_num0'] = np.array(df[numcols_3d])
    X['all_num1'] = np.array(df1[numcols_3d])
    X['all_num2'] = np.array(df2[numcols_3d])
    
    M = np.zeros((df[numcols_3d].shape[0], df[numcols_3d].shape[1], 3), dtype=np.float32)
    M[:,:,0] = X['all_num0']
    M[:,:,1] = X['all_num1']
    M[:,:,2] = X['all_num2']
    
    X['all_num3D'] = M
    
    return X

def train_model(keras_model, X_t, y_train, batch_size, epochs, X_v, y_valid, fold, patience=3):
    early_stopping = EarlyStopping(patience=patience, verbose=1)
    model_checkpoint = ModelCheckpoint("cnn_model_" + str(fold) + ".hdf5",
                                       save_best_only=True, verbose=1, monitor='val_root_mean_squared_error', mode='min')

    hist = keras_model.fit(X_t, y_train, batch_size=batch_size, epochs=epochs,
                            validation_data=(X_v, y_valid), verbose=1,
                            callbacks=[early_stopping, model_checkpoint])

    keras_model = load_model("cnn_model_" + str(fold) + ".hdf5", custom_objects={'root_mean_squared_error': root_mean_squared_error})
    
    return keras_model

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
import gc
# from sklearn.model_selection import GroupKFold
# 

from keras.layers import *
from keras.models import Model
from keras.optimizers import Nadam

oof = np.zeros(len(train))
batch_size = 4096
epochs = 5
models = []


seed = 666

folds = 2
kf = KFold(n_splits=folds, shuffle=False, random_state=seed)

for fold_n, (train_index, valid_index) in enumerate(kf.split(train, target)):
    
# for fold_n, (train_index, valid_index) in enumerate(weirdfolds):
    print('Fold:', fold_n)
    X_train, X_valid = train.iloc[train_index], train.iloc[valid_index]
    y_train, y_valid = target.iloc[train_index], target.iloc[valid_index]
    
    X_train1, X_valid1 = train1.iloc[train_index], train1.iloc[valid_index]
    X_train2, X_valid2 = train2.iloc[train_index], train2.iloc[valid_index]
 
    
    
#     X_train, X_valid = train.iloc[valid_index], train.iloc[train_index]
#     y_train, y_valid = target.iloc[valid_index], target.iloc[train_index]
    
    X_t = get_keras_data(X_train, X_train1, X_train2,  numericals, categoricals)
    X_v = get_keras_data(X_valid, X_valid1, X_valid2, numericals, categoricals)
    del X_train, X_valid, X_train1, X_valid1,X_train2, X_valid2
    gc.collect()
    
    keras_model = model_lofo(dense_dim_1=64, dense_dim_2=64, dense_dim_3=16, dense_dim_4=8, 
                        dropout1=0.2, dropout2=0.1, dropout3=0.1, dropout4=0.1, lr=0.001)
    mod = train_model(keras_model, X_t, y_train, batch_size, epochs, X_v, y_valid, fold_n, patience=3)
    models.append(mod)
    gc.collect()
    print('*'* 50)

Fold: 0
Train on 9585011 samples, validate on 9585012 samples
Epoch 1/5
9585011/9585011 [==============================] - 1727s 180us/step - loss: 0.8785 - root_mean_squared_error: 0.8739 - val_loss: 0.7624 - val_root_mean_squared_error: 0.8689

Epoch 00001: val_root_mean_squared_error improved from inf to 0.86893, saving model to cnn_model_0.hdf5
Epoch 2/5
9585011/9585011 [==============================] - 1681s 175us/step - loss: 0.5003 - root_mean_squared_error: 0.7067 - val_loss: 0.7432 - val_root_mean_squared_error: 0.8580

Epoch 00002: val_root_mean_squared_error improved from 0.86893 to 0.85803, saving model to cnn_model_0.hdf5
Epoch 3/5
6639616/9585011 [===================>..........] - ETA: 6:39 - loss: 0.4598 - root_mean_squared_error: 0.6775

In [65]:
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
import gc
# from sklearn.model_selection import GroupKFold
# 

from keras.layers import *
from keras.models import Model
from keras.optimizers import Nadam

oof = np.zeros(len(train))
batch_size = 4096
epochs = 5
models = []


seed = 666

folds = 2
kf = KFold(n_splits=folds, shuffle=False, random_state=seed)

for fold_n, (train_index, valid_index) in enumerate(kf.split(train, target)):
    
# for fold_n, (train_index, valid_index) in enumerate(weirdfolds):
    print('Fold:', fold_n)
    X_train, X_valid = train.iloc[train_index], train.iloc[valid_index]
    y_train, y_valid = target.iloc[train_index], target.iloc[valid_index]
    
    X_train1, X_valid1 = train1.iloc[train_index], train1.iloc[valid_index]
    X_train2, X_valid2 = train2.iloc[train_index], train2.iloc[valid_index]
 
    
    
#     X_train, X_valid = train.iloc[valid_index], train.iloc[train_index]
#     y_train, y_valid = target.iloc[valid_index], target.iloc[train_index]
    
    X_t = get_keras_data(X_train, X_train1, X_train2,  numericals, categoricals)
    X_v = get_keras_data(X_valid, X_valid1, X_valid2, numericals, categoricals)
    del X_train, X_valid, X_train1, X_valid1,X_train2, X_valid2
    gc.collect()
    
    keras_model = model_lofo(dense_dim_1=64, dense_dim_2=64, dense_dim_3=16, dense_dim_4=8, 
                        dropout1=0.2, dropout2=0.1, dropout3=0.1, dropout4=0.1, lr=0.001)
    mod = train_model(keras_model, X_t, y_train, batch_size, epochs, X_v, y_valid, fold_n, patience=3)
    models.append(mod)
    gc.collect()
    print('*'* 50)

Fold: 0
Train on 9585011 samples, validate on 9585012 samples
Epoch 1/5
9585011/9585011 [==============================] - 1546s 161us/step - loss: 0.7275 - root_mean_squared_error: 0.8118 - val_loss: 0.7963 - val_root_mean_squared_error: 0.8883

Epoch 00001: val_root_mean_squared_error improved from inf to 0.88832, saving model to model_0.hdf5
Epoch 2/5
9585011/9585011 [==============================] - 1539s 161us/step - loss: 0.4636 - root_mean_squared_error: 0.6803 - val_loss: 0.8114 - val_root_mean_squared_error: 0.8963

Epoch 00002: val_root_mean_squared_error did not improve from 0.88832
Epoch 3/5
9585011/9585011 [==============================] - 1539s 161us/step - loss: 0.4281 - root_mean_squared_error: 0.6538 - val_loss: 0.7385 - val_root_mean_squared_error: 0.8554

Epoch 00003: val_root_mean_squared_error improved from 0.88832 to 0.85542, saving model to model_0.hdf5
Epoch 4/5
5480448/9585011 [================>.............] - ETA: 8:59 - loss: 0.4106 - root_mean_squared_err

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



9585011/9585011 [==============================] - 1534s 160us/step - loss: 0.3923 - root_mean_squared_error: 0.6258 - val_loss: 0.7037 - val_root_mean_squared_error: 0.8336

Epoch 00005: val_root_mean_squared_error improved from 0.85542 to 0.83359, saving model to model_0.hdf5
**************************************************
Fold: 1
Train on 9585012 samples, validate on 9585011 samples
Epoch 1/5
9585012/9585012 [==============================] - 1525s 159us/step - loss: 0.7330 - root_mean_squared_error: 0.8051 - val_loss: 0.9465 - val_root_mean_squared_error: 0.9664

Epoch 00001: val_root_mean_squared_error improved from inf to 0.96636, saving model to model_1.hdf5
Epoch 2/5
9585012/9585012 [==============================] - 1516s 158us/step - loss: 0.4480 - root_mean_squared_error: 0.6688 - val_loss: 0.7997 - val_root_mean_squared_error: 0.8884

Epoch 00002: val_root_mean_squared_error improved from 0.96636 to 0.88835, saving model to model_1.hdf5
Epoch 3/5
9585012/9585012 [=======

In [66]:
gc.collect()

0

In [67]:
models

In [68]:
numericals

['square_feet',
 'year_built',
 'air_temperature',
 'cloud_coverage',
 'dew_temperature',
 'precip_depth_1_hr',
 'floor_count']

In [70]:
import gc
# del train, target, X_train, X_valid, y_train, y_valid, X_t, X_v, kf
# gc.collect()

In [71]:
test = pd.read_feather('test_simple_cleanup.feather')

In [72]:
test['square_feet'] = test['square_feet'].apply(lambda x: int(x/1000))

In [73]:
test = test[feat_cols]

In [74]:
feat_cols

['site_id',
 'building_id',
 'primary_use',
 'tm_hour_of_day',
 'tm_day_of_week',
 'meter',
 'square_feet',
 'year_built',
 'air_temperature',
 'cloud_coverage',
 'dew_temperature',
 'precip_depth_1_hr',
 'floor_count',
 'meter_reading_0',
 'meter_reading_1',
 'meter_reading_2',
 'meter_reading_3',
 'cnt_building_per_site',
 'cnt_building_per_site_prim',
 'sqr_mean_per_site',
 'sqr_mean_per_prim_site',
 'RH',
 'heat',
 'windchill',
 'feellike',
 'air_temperature_mean_lag3',
 'dew_temperature_mean_lag3',
 'heat_mean_lag3',
 'windchill_mean_lag3',
 'had_air_temperature',
 'had_cloud_coverage',
 'had_dew_temperature',
 'had_precip_depth_1_hr',
 'had_sea_level_pressure',
 'had_wind_direction',
 'had_wind_speed',
 'had_RH',
 'had_heat',
 'had_windchill',
 'had_feellike',
 'had_air_temperature_mean_lag3',
 'had_dew_temperature_mean_lag3',
 'had_heat_mean_lag3',
 'had_windchill_mean_lag3',
 'had_feellike_mean_lag3']

In [75]:
max_col

{'square_feet': 875,
 'year_built': 2017,
 'air_temperature': 47.20000076293945,
 'cloud_coverage': 9.0,
 'dew_temperature': 35.0,
 'precip_depth_1_hr': 343.0,
 'floor_count': 26,
 'meter_reading_0': 1,
 'meter_reading_1': 1,
 'meter_reading_2': 1,
 'meter_reading_3': 1,
 'cnt_building_per_site': 274,
 'cnt_building_per_site_prim': 92,
 'sqr_mean_per_site': 290625.0,
 'sqr_mean_per_prim_site': 405083.0,
 'RH': 158.5,
 'heat': 225.71316528320312,
 'windchill': 1844.1282958984375,
 'feellike': 225.71316528320312,
 'air_temperature_mean_lag3': 37.233333587646484,
 'dew_temperature_mean_lag3': 34.766666412353516,
 'heat_mean_lag3': 102.4255142211914,
 'windchill_mean_lag3': 1837.2135009765625}

In [76]:
train[feat_cols].mean()

site_id                            8.132886
building_id                      771.497437
primary_use                        3.215099
tm_hour_of_day                    10.871819
tm_day_of_week                     2.887925
meter                              0.654033
square_feet                        0.118790
year_built                         0.391954
air_temperature                    0.334076
cloud_coverage                     0.302423
dew_temperature                    0.222558
precip_depth_1_hr                  0.003364
floor_count                       -0.003319
meter_reading_0                    0.875180
meter_reading_1                    0.561058
meter_reading_2                    0.370234
meter_reading_3                    0.180892
cnt_building_per_site              0.462274
cnt_building_per_site_prim         0.388425
sqr_mean_per_site                  0.227935
sqr_mean_per_prim_site             0.197462
RH                                 0.365671
heat                            

In [78]:

for col in numericals+ fonct_cols_list:
    if col != 'meter_reading':
#         max_ = max(-train[col].min(), train[col].max())
        test[col] = test[col] / max_col[col]


In [79]:
test[feat_cols].mean()

site_id                            8.086134
building_id                      807.582353
primary_use                        3.157983
tm_hour_of_day                    11.500000
tm_day_of_week                     2.998630
meter                              0.664286
square_feet                        0.121675
year_built                         0.399864
air_temperature                    0.327654
cloud_coverage                     0.314451
dew_temperature                    0.216152
precip_depth_1_hr                  0.003423
floor_count                       -0.004202
meter_reading_0                    0.979412
meter_reading_1                    0.564706
meter_reading_2                    0.369748
meter_reading_3                    0.179412
cnt_building_per_site              0.467511
cnt_building_per_site_prim         0.394844
sqr_mean_per_site                  0.245003
sqr_mean_per_prim_site             0.190802
RH                                 0.381577
heat                            

In [86]:
test = reduce_mem_usage(test)

test1 = test.groupby(['building_id', 'meter'])[numcols_3d].shift(1).fillna(0)
test1 = reduce_mem_usage(test1)
test2 = test.groupby(['building_id', 'meter'])[numcols_3d].shift(2).fillna(0)
test2 = reduce_mem_usage(test2)

Memory usage of dataframe is 4573.08 MB
Memory usage after optimization is: 4573.08 MB
Decreased by 0.0%
Memory usage of dataframe is 1590.64 MB
Memory usage after optimization is: 1431.57 MB
Decreased by 10.0%
Memory usage of dataframe is 1590.64 MB
Memory usage after optimization is: 1431.57 MB
Decreased by 10.0%


In [87]:
len(numcols_3d), 

(9,)

In [88]:
from tqdm import tqdm
i=0

res = np.zeros((test.shape[0]),dtype=np.float32)
step_size = 50000
for j in tqdm(range(int(np.ceil(test.shape[0]/step_size)))):
#     for_prediction = get_keras_data(test.iloc[i:i+step_size], numericals, categoricals)
    for_prediction = get_keras_data(test.iloc[i:i+step_size], test1.iloc[i:i+step_size], test2.iloc[i:i+step_size], numericals, categoricals)
    res[i:min(i+step_size,test.shape[0])] = \
       np.expm1(sum([model.predict(for_prediction, batch_size=1024)[:,0] for model in models])/len(models))
    i+=step_size

# for_prediction = get_keras_data(test, test1, test2, numericals, categoricals)
# res = np.expm1(sum([model.predict(for_prediction, batch_size=1024)[:,0] for model in models])/len(models))


 37%|███▋      | 309/834 [17:14<29:11,  3.34s/it]


 74%|███████▍  | 617/834 [34:28<12:07,  3.35s/it]


100%|██████████| 834/834 [46:37<00:00,  3.30s/it]

In [89]:
res

array([186.86758  ,  74.845665 ,   5.710334 , ...,   6.9682903,
       179.84003  ,   3.8823416], dtype=float32)

In [90]:
submission = pd.read_csv('sample_submission.csv')
submission['meter_reading'] = res
submission.loc[submission['meter_reading']<0, 'meter_reading'] = 0

submission

,row_id,meter_reading
0,0,186.867584
1,1,74.845665
2,2,5.710334
3,3,337.059570
4,4,1659.818359
...,...,...
41697595,41697595,7.096662
41697596,41697596,4.216076
41697597,41697597,6.968290
41697598,41697598,179.840027


In [91]:
submission

,row_id,meter_reading
0,0,186.867584
1,1,74.845665
2,2,5.710334
3,3,337.059570
4,4,1659.818359
...,...,...
41697595,41697595,7.096662
41697596,41697596,4.216076
41697597,41697597,6.968290
41697598,41697598,179.840027


In [140]:
1

1

In [92]:
submission.to_csv('submission_nn007lofo.csv.gz', index=False, compression='gzip', float_format='%.4f')

In [93]:
!kaggle competitions submit -c ashrae-energy-prediction -f submission_nn007lofo.csv.gz -m "keras-nn-with-embeddings-for-cat-features-1-12-originalc1-nnlofo-1-14-Copy1"



100%|████████████████████████████████████████| 261M/261M [00:05<00:00, 46.4MB/s]
Successfully submitted to ASHRAE - Great Energy Predictor III

In [33]:
1+1

2